In [1]:
# pip install crypten --user
import crypten

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from model import Net
import time

2022-11-17 23:11:27.320695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/local/lib::/.singularity.d/libs
2022-11-17 23:11:27.320733: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/zj6nw/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/zj6nw/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c106IValue23reportToTensorTypeErrorEv
  warn(f"Failed to load image Python extension: {e}")


ModuleNotFoundError: No module named 'torch.ao'

In [2]:
torch.cuda.get_device_name(device=0)

'NVIDIA A100-SXM4-80GB'

In [3]:
use_cuda = torch.cuda.is_available()
# use_mps = torch.backends.mps.is_available()
if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

batch_size=64
test_batch_size=1000
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset = datasets.MNIST('../data', train=False,
                   transform=transform)
test_loader = torch.utils.data.DataLoader(dataset, **test_kwargs)

In [4]:
# import crypten
# import torch

crypten.init()
torch.set_num_threads(1)

In [5]:
from tqdm.notebook import tqdm
import crypten.mpc as mpc
import crypten.communicator as comm

# @mpc.run_multiprocess(world_size=2)
def get_time_elapsed_crypten(device, test_loader):
    plaintext_model = torch.load('./vanilla_pytorch_mnist_'+f'{torch.cuda.get_device_name(0)}.pth').to('cpu')
    dummy_input = torch.empty((1, 1, 28, 28))

    private_model = crypten.nn.from_pytorch(plaintext_model, dummy_input)
    private_model.encrypt(src=0)
    private_model.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        t0 = time.perf_counter()
        for data, target in tqdm(test_loader):
            target = target
            data_enc = crypten.cryptensor(data)
            output_enc = private_model(data_enc)
            output = output_enc.get_plain_text()
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        time_elapsed = time.perf_counter() - t0

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print('Time Elapsed:{}'.format(time_elapsed))
    return time_elapsed

time=get_time_elapsed_crypten( device, test_loader)

/usr/local/lib/python3.8/site-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/home/ss4yd/.local/lib/python3.8/site-packages/crypten/nn/onnx_converter.py:161: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  param = torch.from_numpy(numpy_helper.to_array(node))


  0%|          | 0/10 [00:00<?, ?it/s]

/home/ss4yd/.local/lib/python3.8/site-packages/crypten/encoder.py:75: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dividend = tensor // self._scale - correction



Test set: Average loss: 0.1648, Accuracy: 9516/10000 (95%)

Time Elapsed:1426.016528827


In [6]:
import pandas as pd
df=pd.DataFrame()
df['device']=[torch.cuda.get_device_name(0)]
df['time']=[time]
df.to_csv('crypten_time_elapsed_'+f'{torch.cuda.get_device_name(0)}.csv', index=False)
df

,device,time
0,NVIDIA A100-SXM4-80GB,1426.016529


In [7]:
df['device']=[torch.cuda.get_device_name(0)]
df['time']=[time]
df

,device,time
0,NVIDIA A100-SXM4-80GB,1426.016529


In [8]:
torch.cuda.get_device_name(0)

'NVIDIA A100-SXM4-80GB'